In [ ]:
import random
import numpy as np

size = 6
training_number = 500
alpha = 0.5 #learning rate
epsilon = 0.5 #probability of exploration
gamma = 0.7 #how much future rewards are valued
max_tries = 50
#movement 1 = up, 2 = down, 3 = left, 4 = right
actions = [1, 2, 3, 4]

#generates map
def create_map(size):
  matrix_map = []
  for x in range(size):
    matrix_map.append([-1] * size)

#generates q table
def q_table_init(size):
  q_table = []
  matrix_size = size ** 2
  for x in range(matrix_size):
    q_table.append([0, 0, 0, 0])
  return q_table

#randomly generates a starting point for the player
def player_starting_point(matrix_map, state_number):
  starting_point = [random.randint(0, len(matrix_map) -1), random.randint(0, state_number-1)]
  matrix_map[starting_point[0]][starting_point[1]] = 0
  return starting_point

#randomly generates a starting point for wumpus
def wumpus_starting_point(matrix_map, state_number):
  wumpus_point = [random.randint(0, len(matrix_map) -1), random.randint(0, state_number-1)]
  while matrix_map[wumpus_point[0]][wumpus_point[1]] == 0:
    #makes sure wumpus is not generated on top of the player
    wumpus_point = [random.randint(0, len(matrix_map) -1), random.randint(0, state_number-1)]
  matrix_map[wumpus_point[0]][wumpus_point[1]] = 1000
  return wumpus_point

#randomly generates a series of pit falls
def generate_holes(number, matrix_map, state_number):
  hole_locations = []
  for x in range(number):
    hole_point = [random.randint(0, len(matrix_map) - 1), random.randint(0, state_number - 1)]
    while matrix_map[hole_point[0]][hole_point[1]] == 0 or matrix_map[hole_point[0]][hole_point[1]] == 1000 or matrix_map[hole_point[0]][hole_point[1]] == -100:
      hole_point = [random.randint(0, len(matrix_map) - 1), random.randint(0, state_number - 1)]
      hole_locations.append(hole_point)
    matrix_map[hole_point[0]][hole_point[1]] = -100

  return hole_locations


def action_take(matrix_map, q_table, current_location, action, size):
  if current_location[0] == len(size) - 1 and current_location[1] == 0 and action == 1 or action == 3:
    #This shows that the current place of the user is in the left most part of the matrix
    return -200, current_location
  elif current_location[0] < len(size) - 1 and current_location[1] == len(size) - 1 and action == 1 or action == 4:
    #This shows that the current place of the user is in the right most part of the matrix
    return -200, current_location
  elif current_location[0] == 0 and current_location[1] < len(size) - 1 and action == 1:
    #This shows that the current place of the user is in the upper most part of the matrix
    return -200, current_location
  elif current_location[0] == len(size) - 1 and current_location[1] < len(size) - 1 and action == 2:
    #This shows that the current place of the user is in the lower most part of the matrix
    return -200, current_location
  else:
    #This just returns the current user location
    if action == 1:
      new_location = [current_location[0] - 1, current_location[1]]
      return matrix_map[new_location[0]][new_location[1]], new_location
    elif action == 2:
      new_location = [current_location[0] + 1][current_location[1]]
      return matrix_map[new_location[0]][new_location[1]], new_location
    elif action == 3:
      new_location = [current_location[0]][current_location[1] - 1]
      return matrix_map[new_location[0]][new_location[1]], new_location
    elif action == 4:
      new_location = [current_location[0]][current_location[1] + 1]
      return matrix_map[new_location[0]][new_location[1]], new_location
  

def convert_matrix_to_q_table(current_location, q_table):
  q_table_location = current_location[0] * 5 + current_location[0] + current_location[1]
  return q_table_location


def q_learning(player_current_location, action, matrix_map, q_table, state_number, wumpus_point, hole_point, size=size, alpha=alpha, epsilon=epsilon, gamma=gamma, training_number=training_number, max_tries=max_tries):
  rewards = []

  for x in range(training_number):
    reward = 0
    current_location = player_current_location
    q_table_location = convert_matrix_to_q_table(current_location, q_table)
    
    for y in range(max_tries):
      if random.uniform(0, 1) > epsilon:
        action_taken = random.randint(0, state_number - 1)
      else:
        action_taken = np.argmax(q_table[q_table_location])
        for z in range(state_number):
          if q_table[q_table_location][z] == action_taken:
            if z == 0:
              action = 1
              break
            elif z == 1:
              action = 2
              break
            elif z == 2:
              action = 3
              break
            elif z == 3:
              action = 4
              break
      
      reward, new_location = action_take(matrix_map, q_table, current_location, action, size)
      rewards.append(reward)
      q_table_location = convert_matrix_to_q_table(new_location, q_table)
      q_table[q_table_location][action] = (1 - alpha) * q_table[q_table_location][action] + alpha * (reward + gamma * max(q_table[q_table_location]))
      if new_location == wumpus_point or new_location in hole_point:
        break
      else:
        current_location = new_location
    return rewards, q_table